<a href="https://colab.research.google.com/github/Ucchwas/ECG-Classification-using-CNN/blob/main/ANN_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import SGD
import seaborn as sns
#%matplotlib inline
import warnings
# program to compute the time
# of execution of any python code
from datetime import datetime
from requests.packages import target

warnings.filterwarnings('ignore')

In [43]:
start = datetime.now()

df = pd.read_csv('/content/drive/MyDrive/Datasets/mitbih_train_Ihua.csv')

In [44]:
df.drop('1',axis='columns',inplace=True)
#print(df.dtypes)
df.fillna(0, inplace=True)
X = df.drop('Label1',axis='columns')
#Xs = df.drop('Label1',axis='columns')
#X = Xs[df['Label1'] != 1 & 2 & 3]
y = testLabels = df.Label1.astype(np.float32)
#ys = testLabels = df.Label1.astype(np.float32)
#y = ys[df['Label1'] != 1 & 2 & 3]

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2,train_size=0.8, random_state=15, stratify=y)
#print(y_train.value_counts())
#print(y.value_counts())
#print(y.value_counts())
#print(len(X_train.columns))

#from tensorflow_addons import losses
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [46]:
X_train.shape

(70044, 186)

In [47]:
X_test.shape

(17511, 186)

In [48]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):

    model = keras.Sequential([
        keras.layers.Dense(256, input_dim=186, activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(5, activation='softmax'),
    ])

    # Define optimizer with custom learning rate
    optimizer = keras.optimizers.Adam(learning_rate=0.001)

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    # Define class weights if provided
    class_weights = None
    if weights != -1:
        class_weights = weights

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, class_weight=class_weights)

    # Evaluate on test data
    print(model.evaluate(X_test, y_test))

    # Predict on training data
    y_train_preds = model.predict(X_train)
    y_train_preds = np.argmax(y_train_preds, axis=1)

    # Print classification report on training data
    print("Classification Report (Training Data):\n", classification_report(y_train, y_train_preds))

    return model, y_train_preds

In [49]:
model, y_preds = ANN(X_train, y_train, X_test, y_test, 'sparse_categorical_crossentropy', -1)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_preds)
print("Cconfusion Matrix: \n",cm)

Epoch 1/50
2189/2189 [==============================] - 11s 5ms/step - loss: 0.2248 - accuracy: 0.9371
Epoch 2/50
2189/2189 [==============================] - 10s 4ms/step - loss: 0.1386 - accuracy: 0.9611
Epoch 3/50
2189/2189 [==============================] - 9s 4ms/step - loss: 0.1137 - accuracy: 0.9679
Epoch 4/50
2189/2189 [==============================] - 11s 5ms/step - loss: 0.0994 - accuracy: 0.9716
Epoch 5/50
2189/2189 [==============================] - 10s 5ms/step - loss: 0.0894 - accuracy: 0.9742
Epoch 6/50
2189/2189 [==============================] - 9s 4ms/step - loss: 0.0823 - accuracy: 0.9752
Epoch 7/50
2189/2189 [==============================] - 10s 5ms/step - loss: 0.0746 - accuracy: 0.9770
Epoch 8/50
2189/2189 [==============================] - 10s 5ms/step - loss: 0.0691 - accuracy: 0.9783
Epoch 9/50
2189/2189 [==============================] - 9s 4ms/step - loss: 0.0649 - accuracy: 0.9795
Epoch 10/50
2189/2189 [==============================] - 11s 5ms/step - loss

In [50]:
model.save("ANN_Multiclass.h5")

In [51]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("ANN_Multiclass.h5")

In [52]:
# convert to tensorflow lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("ANN_Multiclass.tflite", "wb").write(tflite_model)

461960